In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import re
import scipy.stats as stats

In [2]:
%cd C:\Users\vikas\OneDrive\Documents\Bussiness Analytics\Python\Assignment\Basic Statistics - Hypothesis Testing

C:\Users\vikas\OneDrive\Documents\Bussiness Analytics\Python\Assignment\Basic Statistics - Hypothesis Testing


In [3]:
treatment_facility=pd.read_csv("Treatment_Facility.csv")
priority_assessment=pd.read_csv("Priority_Assessment.csv")
price_quotes=pd.read_csv("Price_Quotes.csv")
loan_data=pd.read_csv("LoansData.csv")
films=pd.read_csv("Films.csv")

In [4]:
loan_data.columns= [col.replace('.','_') for col in loan_data.columns]
loan_data["Debt_To_Income_Ratio"]=loan_data["Debt_To_Income_Ratio"].str.replace("%","").astype(float)
loan_data["Interest_Rate"]=loan_data["Interest_Rate"].str.replace("%","").astype(float)
loan_data['FICO_SCORE'] = ((loan_data.FICO_Range.str.split('-', expand=True)[0].astype(float)) + 
                    (loan_data.FICO_Range.str.split('-', expand=True)[1].astype(float)))/2
loan_data.columns= [col.replace('.','_') for col in loan_data.columns]

In [5]:
def missing_treat(x):
    if ((x.dtype=='float') or (x.dtype=='int')):
        x = x.fillna(x.median())
    elif x.dtype == 'object':
        x = x.fillna(x.mode()[0])
    else:
        x
    return(x)

In [6]:
loan_data=loan_data.apply(missing_treat)

In [7]:
loan_data

,Amount_Requested,Amount_Funded_By_Investors,Interest_Rate,Loan_Length,Loan_Purpose,Debt_To_Income_Ratio,State,Home_Ownership,Monthly_Income,FICO_Range,Open_CREDIT_Lines,Revolving_CREDIT_Balance,Inquiries_in_the_Last_6_Months,Employment_Length,FICO_SCORE
0,20000.0,20000.00,8.90,36 months,debt_consolidation,14.90,SC,MORTGAGE,6541.67,735-739,14.0,14272.0,2.0,< 1 year,737.0
1,19200.0,19200.00,12.12,36 months,debt_consolidation,28.36,TX,MORTGAGE,4583.33,715-719,12.0,11140.0,1.0,2 years,717.0
2,35000.0,35000.00,21.98,60 months,debt_consolidation,23.81,CA,MORTGAGE,11500.00,690-694,14.0,21977.0,1.0,2 years,692.0
3,10000.0,9975.00,9.99,36 months,debt_consolidation,14.30,KS,MORTGAGE,3833.33,695-699,10.0,9346.0,0.0,5 years,697.0
4,12000.0,12000.00,11.71,36 months,credit_card,18.78,NJ,RENT,3195.00,695-699,11.0,14469.0,0.0,9 years,697.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,30000.0,29950.00,16.77,60 months,debt_consolidation,19.23,NY,MORTGAGE,9250.00,705-709,15.0,45880.0,1.0,8 years,707.0
2496,16000.0,16000.00,14.09,60 months,home_improvement,21.54,MD,OWN,8903.25,740-744,18.0,18898.0,1.0,10+ years,742.0
2497,10000.0,10000.00,13.99,36 months,debt_consolidation,4.89,PA,MORTGAGE,2166.67,680-684,4.0,4544.0,0.0,10+ years,682.0
2498,6000.0,6000.00,12.42,36 months,major_purchase,16.66,NJ,RENT,3500.00,675-679,8.0,7753.0,0.0,5 years,677.0


# BUSINESS PROBLEM-1

## a. Intrest rate is varied for different loan amounts (Less intrest charged for high loan amounts)

#### stage:1
#### HO : intrest_rate == amount
#### H1 : intrest_rate <> amount

In [8]:
stats.pearsonr(loan_data.Amount_Requested,loan_data.Interest_Rate)

(0.3324401743401226, 1.4231505725301343e-65)

#### stage 4 - conclusion
#### we reject the null hypothesis,there is proportion

## b. Loan length is directly effecting intrest rate.

#### stage:1
#### HO : mon_36.Interest_Rate == mon_60.Interest_Rate
#### H1 : mon_36.Interest_Rate <> mon_60.Interest_Rate

In [9]:
mon_36=loan_data[loan_data.Loan_Length=="36 months"]
mon_60=loan_data[loan_data.Loan_Length=="60 months"]
stats.f_oneway(mon_36.Interest_Rate,mon_60.Interest_Rate)

F_onewayResult(statistic=548.0367191411436, pvalue=9.605991607837365e-110)

In [10]:
mon_36=loan_data[loan_data.Loan_Length=="36 months"]
mon_60=loan_data[loan_data.Loan_Length=="60 months"]
stats.f_oneway(mon_36.Interest_Rate,mon_60.Interest_Rate)

F_onewayResult(statistic=548.0367191411436, pvalue=9.605991607837365e-110)

#### stage 4 -conclusion- we reject the null hypothesis,because there is effect in intrest rate on changing the loan lenght

## c. Intrest rate varies for different purpose of loans

#### stage:1
#### HO : a=b=c=d=e..
#### H1 : either one is different

In [11]:
a=loan_data[loan_data.Loan_Purpose=="debt_consolidation"]
b=loan_data[loan_data.Loan_Purpose=="credit_card"]
c=loan_data[loan_data.Loan_Purpose=="other"]
d=loan_data[loan_data.Loan_Purpose=="moving"]
e=loan_data[loan_data.Loan_Purpose=="car"]
f=loan_data[loan_data.Loan_Purpose=="vacation"]
g=loan_data[loan_data.Loan_Purpose=="home_improvement"]
h=loan_data[loan_data.Loan_Purpose=="house"]
i=loan_data[loan_data.Loan_Purpose=="major_purchase"]
j=loan_data[loan_data.Loan_Purpose=="educational"]
k=loan_data[loan_data.Loan_Purpose=="medical"]
l=loan_data[loan_data.Loan_Purpose=="small_business"]
m=loan_data[loan_data.Loan_Purpose=="renewable_energy"]
n=loan_data[loan_data.Loan_Purpose=="wedding"]
stats.f_oneway(a.Interest_Rate,b.Interest_Rate,c.Interest_Rate,d.Interest_Rate,e.Interest_Rate,
               f.Interest_Rate,g.Interest_Rate,h.Interest_Rate,i.Interest_Rate,j.Interest_Rate,
               k.Interest_Rate,l.Interest_Rate,m.Interest_Rate,n.Interest_Rate)

F_onewayResult(statistic=7.482212567168055, pvalue=1.1627206793499994e-14)

#### stage 4 - conclusion we reject the null hypothesis,yes there is effect in intrest rate

## d. There is relationship between FICO scores and Home Ownership. It means that, People with owning home will have high FICO scores.

#### stage:1
#### HO : a=b=c=d=e....
#### H1 : either one is different 

In [12]:
a=loan_data[loan_data.Home_Ownership=="MORTGAGE"]
b=loan_data[loan_data.Home_Ownership=="RENT"]
c=loan_data[loan_data.Home_Ownership=="OWN"]
d=loan_data[loan_data.Home_Ownership=="OTHER"]
e=loan_data[loan_data.Home_Ownership=="NONE"]

stats.f_oneway(a.FICO_SCORE,b.FICO_SCORE,c.FICO_SCORE,d.FICO_SCORE,e.FICO_SCORE)

F_onewayResult(statistic=19.345822938089942, pvalue=1.0902432079019346e-15)

#### stage 4 conclusion - we reject the null hypothesis,yes there is effect in FICO scores because of Ownership

# BUSINESS PROBLEM - 2

### BUSINESS PROBLEM: We would like to assess if there is any difference in the average price quotes provided by Mary and Barry.

#### stage:1
#### HO : avarage of mary == avarage of barry
#### H1 : avarage of mary <> avarage of barry

In [13]:
stats.ttest_ind(price_quotes.Mary_Price,price_quotes.Barry_Price)

Ttest_indResult(statistic=-1.4147436739281787, pvalue=0.17114226132118285)

#### stage 4 conclusion - We failed to recjected null hypothesis,there is a no difference between price quotes

# BUSINESS PROBLEM-3

## BUSINESS PROBLEM: Determine what effect, if any, the reengineering effort had on the incidence behavioral problems and staff turnover. i.e To determine if the reengineering effort changed the critical incidence rate. Is there evidence that the critical incidence rate improved?

#### stage:1
#### HO : Prior_staff turnover == Post_staff turnover
#### H1 : Prior_staff turnover <> Post_staff turnover

In [18]:
Prior_staff=treatment_facility[treatment_facility.Reengineer=="Prior"]
Post_staff=treatment_facility[treatment_facility.Reengineer=="Post"]
stats.ttest_ind(Prior.Employee_Turnover,Post.Employee_Turnover)

Ttest_indResult(statistic=-1.770306704753604, pvalue=0.09361109345535304)

#### stage 4 conclusion - with 90% confidence we can reject the null hypothesis,there is a effect on employee turnover

#### stage:1
#### HO : Prior_staff turnover == Post_staff turnover
#### H1 : Prior_staff turnover <> Post_staff turnover

In [20]:
Prior_staff1=treatment_facility[treatment_facility.Reengineer=="Prior"]
Post_staff1=treatment_facility[treatment_facility.Reengineer=="Post"]
stats.ttest_ind(Prior_staff1.VAR5,Post_staff1.VAR5)

Ttest_indResult(statistic=1.6279144253528646, pvalue=0.12091989189884149)

#### stage 4 conclusion - we are failed to reject the null hypothesis,we can't say that their is change in critical incidence

# BUSINESS PROBLEM-4

## BUSINESS PROBLEM: We will focus on the prioritization system. If the system is working, then high priority jobs, on average, should be completed more quickly than medium priority jobs, and medium priority jobs should be completed more quickly than low priority jobs. Use the data provided to determine whether this is, in fact, occurring.

#### stage:1
#### HO : high_priority == medium_priority == low_priority
#### H1 : high_priority <> medium_priority <> low_priority

In [21]:
high=priority_assessment[priority_assessment.Priority=="High"]
medium=priority_assessment[priority_assessment.Priority=="Medium"]
low=priority_assessment[priority_assessment.Priority=="Low"]
stats.f_oneway(high.Days,medium.Days,low.Days)

F_onewayResult(statistic=1.812311010076072, pvalue=0.16411459461716182)

#### stage 4 conclusion - we failed to reject the null hypothesis,there no change in with respect to priority

# BUSINESS PROBLEM-5

## a.What is the overall level of customer satisfaction?

#### stage:1
#### HO : customer satisfaction == 5
#### H1 : customer satisfaction <> 5

In [25]:
films=films.apply(missing_treat)

In [26]:
stats.ttest_1samp(films.Overall,5)

Ttest_1sampResult(statistic=-89.23581000259827, pvalue=1.290517734123711e-232)

In [27]:
films["Overall"].mean()

1.621212121212121

#### stage 4 conclusion - we reject the null hypothesis,we can say that customer satisfaction is different from 5

## b.What factors are linked to satisfaction?

#### stage:1
#### HO : rating_of_Sinage == rating_of_Parking == rating_of_Clean
#### H1 : rating_of_Sinage <> rating_of_Parking <> rating_of_Clean

In [28]:
films.corr()

,_rowstate_,Sinage,Parking,Clean,Overall,Age,Income
_rowstate_,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sinage,NaN,1.000000,0.469180,0.349144,0.382103,-0.089461,-0.039997
Parking,NaN,0.469180,1.000000,0.440771,0.514429,-0.023468,-0.025945
Clean,NaN,0.349144,0.440771,1.000000,0.350822,-0.000114,0.018345
Overall,NaN,0.382103,0.514429,0.350822,1.000000,-0.032904,-0.003734
Age,NaN,-0.089461,-0.023468,-0.000114,-0.032904,1.000000,0.135642
Income,NaN,-0.039997,-0.025945,0.018345,-0.003734,0.135642,1.000000


In [ ]:
cross1=pd.crosstab(flim.Sinage,flim.Overall)
stats.chi2_contingency(cross1)

In [29]:
cross1=pd.crosstab(films.Sinage,films.Overall)
stats.chi2_contingency(cross1)

(125.61859671027881,
 4.53298970003405e-19,
 16,
 array([[5.17060606e+01, 5.54727273e+01, 4.10909091e+00, 3.42424242e-01,
         1.36969697e+00],
        [7.22969697e+01, 7.75636364e+01, 5.74545455e+00, 4.78787879e-01,
         1.91515152e+00],
        [2.19636364e+01, 2.35636364e+01, 1.74545455e+00, 1.45454545e-01,
         5.81818182e-01],
        [2.74545455e+00, 2.94545455e+00, 2.18181818e-01, 1.81818182e-02,
         7.27272727e-02],
        [2.28787879e+00, 2.45454545e+00, 1.81818182e-01, 1.51515152e-02,
         6.06060606e-02]]))

In [30]:
cross2=pd.crosstab(films.Parking,films.Overall)
stats.chi2_contingency(cross2)

(230.61691663391676,
 4.783162283608494e-40,
 16,
 array([[6.17727273e+01, 6.62727273e+01, 4.90909091e+00, 4.09090909e-01,
         1.63636364e+00],
        [7.09242424e+01, 7.60909091e+01, 5.63636364e+00, 4.69696970e-01,
         1.87878788e+00],
        [1.32696970e+01, 1.42363636e+01, 1.05454545e+00, 8.78787879e-02,
         3.51515152e-01],
        [3.20303030e+00, 3.43636364e+00, 2.54545455e-01, 2.12121212e-02,
         8.48484848e-02],
        [1.83030303e+00, 1.96363636e+00, 1.45454545e-01, 1.21212121e-02,
         4.84848485e-02]]))

In [31]:
cross3=pd.crosstab(films.Clean,films.Overall)
stats.chi2_contingency(cross3)

(127.71945555999517,
 1.7772535915434025e-19,
 16,
 array([[6.58909091e+01, 7.06909091e+01, 5.23636364e+00, 4.36363636e-01,
         1.74545455e+00],
        [7.09242424e+01, 7.60909091e+01, 5.63636364e+00, 4.69696970e-01,
         1.87878788e+00],
        [1.09818182e+01, 1.17818182e+01, 8.72727273e-01, 7.27272727e-02,
         2.90909091e-01],
        [2.28787879e+00, 2.45454545e+00, 1.81818182e-01, 1.51515152e-02,
         6.06060606e-02],
        [9.15151515e-01, 9.81818182e-01, 7.27272727e-02, 6.06060606e-03,
         2.42424242e-02]]))

#### stage 4 conclusion - we reject the null hypothesis,we can say that all factors are different

## c.What is the demographic profile of Film on the Rocks patrons?

In [32]:
pd.crosstab(films.Movie,films.Sinage)

Sinage,1.0,2.0,3.0,4.0,5.0
Movie,,,,,
Ferris Buellers Day Off,53,64,19,0,1
Old School,12,13,4,2,1
Willy Wonka,48,81,25,4,3


In [33]:
pd.crosstab(films.Movie,films.Parking)

Parking,1.0,2.0,3.0,4.0,5.0
Movie,,,,,
Ferris Buellers Day Off,54,66,11,5,1
Old School,17,11,3,0,1
Willy Wonka,64,78,15,2,2


In [34]:
pd.crosstab(films.Movie,films.Clean)

Clean,1.0,2.0,3.0,4.0,5.0
Movie,,,,,
Ferris Buellers Day Off,61,62,12,2,0
Old School,12,16,3,0,1
Willy Wonka,71,77,9,3,1


In [35]:
pd.crosstab(films.Movie,films.Gender)

Gender,1,2,Female,Male
Movie,,,,
Ferris Buellers Day Off,38,67,19,13
Old School,13,19,0,0
Willy Wonka,53,108,0,0


In [36]:
pd.crosstab(films.Movie,films.Marital_Status)

Marital_Status,1,2,Married,Single,Slngle
Movie,,,,,
Ferris Buellers Day Off,29,76,12,18,2
Old School,11,21,0,0,0
Willy Wonka,48,113,0,0,0


In [37]:
pd.crosstab(films.Movie,films.Overall)

Overall,1.0,2.0,3.0,4.0,5.0
Movie,,,,,
Ferris Buellers Day Off,59,71,4,1,2
Old School,21,10,0,0,1
Willy Wonka,71,81,8,0,1


## d.In what media outlet(s) should the film series be advertised?

In [38]:
to_add=pd.crosstab(films.Movie,films.Hear_About)
to_add

Hear_About,1,"1,5",2,"2,5",3,"3,4","3,5",4,"4,5",5,"5,4"
Movie,,,,,,,,,,,
Ferris Buellers Day Off,13,1,4,1,4,0,0,17,1,96,0
Old School,1,0,4,0,1,1,0,1,0,24,0
Willy Wonka,8,0,4,1,9,1,1,23,0,113,1
